In [18]:
import tensorflow as tf
import numpy as np
import spout

print(tf.__version__)

2.2.0


In [2]:
INPUT_TENSOR_NAME = 'ImageTensor:0'
OUTPUT_TENSOR_NAME = 'SemanticPredictions:0'
INPUT_SIZE = 513
MODEL = './models/deeplabv3_mnv2_pascal_train_aug/frozen_inference_graph.pb'

In [3]:
def wrap_frozen_graph(graph_def, inputs, outputs):
    def _imports_graph_def():
        tf.compat.v1.import_graph_def(graph_def, name="")
    wrapped_import = tf.compat.v1.wrap_function(_imports_graph_def, [])
    import_graph = wrapped_import.graph
    return wrapped_import.prune(
        tf.nest.map_structure(import_graph.as_graph_element, inputs),
        tf.nest.map_structure(import_graph.as_graph_element, outputs))

In [4]:
def create_pascal_label_colormap():
    """Creates a label colormap used in PASCAL VOC segmentation benchmark.
    Returns:
    A Colormap for visualizing segmentation results.
    """
    colormap = np.zeros((256, 3), dtype=int)
    ind = np.arange(256, dtype=int)
    for shift in reversed(range(8)):
        for channel in range(3):
            colormap[:, channel] |= ((ind >> channel) & 1) << shift
            ind >>= 3
    return colormap
def label_to_color_image(label):
    """Adds color defined by the dataset colormap to the label.
    Args:
    label: A 2D array with integer type, storing the segmentation label.
    Returns:
    result: A 2D array with floating type. The element of the array
            is the color indexed by the corresponding element in the input label
            to the PASCAL color map.
    Raises:
        ValueError: If label is not of rank 2 or its value is larger than color
        map maximum entry.
    """
    if label.ndim != 2:
        raise ValueError('Expect 2-D input label')
    colormap = create_pascal_label_colormap()
    if np.max(label) >= len(colormap):
        raise ValueError('label value too large.')
    return colormap[label]

LABEL_NAMES = np.asarray([
    'background', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus',
    'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike',
    'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tv'
])

#FULL_LABEL_MAP = np.arange(len(LABEL_NAMES)).reshape(len(LABEL_NAMES), 1)
#FULL_COLOR_MAP = label_to_color_image(FULL_LABEL_MAP)

In [5]:
# Load frozen graph using TensorFlow 1.x functions
with tf.io.gfile.GFile(MODEL, "rb") as f:
    graph_def = tf.compat.v1.GraphDef()
    loaded = graph_def.ParseFromString(f.read())
    
# Wrap frozen graph to ConcreteFunctions
frozen_func = wrap_frozen_graph(graph_def=graph_def, inputs=[INPUT_TENSOR_NAME], outputs=[OUTPUT_TENSOR_NAME])

In [17]:
# Create PySpout Instance
sp = spout.PySpout()

while(1):
    image = sp.receive('test')[:,:,1:]

    # pre-processing
    width, height = image.shape[:2]
    resize_ratio = 1.0 * INPUT_SIZE / max(width, height)
    target_size = (int(resize_ratio * width), int(resize_ratio * height))
    resized_image = tf.image.resize_with_pad(image, target_size[0], target_size[1], method='nearest', antialias=True)
    img_tensor = tf.convert_to_tensor(resized_image)
    img_final = tf.expand_dims(img_tensor, 0)

    # prediction
    seg_map = frozen_func(ImageTensor=img_final)[0]
    
    # post-processing
    seg_map = tf.tile(tf.expand_dims(seg_map[0], 2), [1,1,3])
    seg_map = seg_map.numpy().astype(np.uint8)

    sp.send('seg', seg_map)

KeyboardInterrupt: 